Rebuild the original ResNet50 architecture to get an understanding of the building blocks

In [14]:
import tensorflow as tf
import keras
from keras import Input, layers
from keras.layers import Input, Add, Dense, Flatten, Activation
from keras.layers import ZeroPadding2D, BatchNormalization, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import LSTM, Bidirectional
from keras.models import Model, load_model

from resnets_utils import *

print(tf.__version__)
print(keras.__version__)

2.1.0
2.3.1


# ResNet50 

## Tests

In [2]:
#simple example from book
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu', name='first_layer')(input_tensor)
x = layers.Dense(32, activation='relu', name='second_layer')(x)
output_tensor = layers.Dense(10, activation='softmax', name='output_layer')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
first_layer (Dense)          (None, 32)                2080      
_________________________________________________________________
second_layer (Dense)         (None, 32)                1056      
_________________________________________________________________
output_layer (Dense)         (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [7]:
input_tensor = Input(shape=(64,64,))
x = Dense(32, activation='relu', name='first_layer')(input_tensor)
x = Dense(32, activation='relu', name='second_layer')(x)
x = LSTM(16, return_sequences=True)(x)
x = Bidirectional(LSTM(16))(x)
output_tensor = Dense(10, activation='softmax', name='output_layer')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64)            0         
_________________________________________________________________
first_layer (Dense)          (None, 64, 32)            2080      
_________________________________________________________________
second_layer (Dense)         (None, 64, 32)            1056      
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 16)            3136      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32)                4224      
_________________________________________________________________
output_layer (Dense)         (None, 10)                330       
Total params: 10,826
Trainable params: 10,826
Non-trainable params: 0
_______________________________________________________

In [8]:
#get implemented resnet50 from keras
from keras.applications.resnet50 import ResNet50
resnet50_keras = ResNet50(weights=None)
resnet50_keras.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/Residual_Networks_yourself.ipynb <br>
https://arxiv.org/pdf/1512.03385.pdf <br>
https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py

## Rebuild Model

In [9]:
# Identity Block
def identity_block(input_x, kernel_size, filters, stage, block):
    """
    Implementation of the identity block (or bottleneck block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv2D(filters = filters1,
               kernel_size = (1,1),
               strides = (1,1),
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv2D(filters = filters2,
               kernel_size = kernel_size,
               strides = (1,1),
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv2D(filters = filters3,
               kernel_size = (1,1),
               strides = (1,1),
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2c')(x)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [10]:
# Convolutional Block
def convolutional_block(input_x, kernel_size, filters, stage, block, strides=(2, 2)):
    """
    Implementation of the convolutional (or standard block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        stride: strides for the first convolutional layer in the block
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv2D(filters = filters1,
               kernel_size = (1,1),
               strides = strides,
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv2D(filters = filters2,
               kernel_size = kernel_size,
               strides = (1,1),
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv2D(filters = filters3,
               kernel_size = (1,1),
               strides = (1,1),
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(axis = 3,
                           name = bn_name_base + '2c')(x)
    
    #shortcut path
    x_shortcut = Conv2D(filters = filters3,
                        kernel_size = (1,1),
                        strides = strides,
                        kernel_initializer = 'he_normal',
                        name = conv_name_base + '1')(x_shortcut)
    x_shortcut = BatchNormalization(axis = 3,
                                    name = bn_name_base + '1')(x_shortcut)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [11]:
#put the model together
def ResNet50(input_shape=(64, 64, 3), classes=6):
    """
    Instantiate the ResNet50 architecture
    
    # Arguments
        input_shape: shape of the input data tuple
        classes: number of classes to classify images into
        
    # Returns
        A Keras model instance.
    """
    
    #define input to the model as a tensor with the shape input_shape
    x_input = Input(input_shape)
    
    #Zero Padding
    x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(x_input)
    
    #Stage 1
    x = Conv2D(64, (7,7), 
               strides = (2,2),
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = 'conv1')(x)
    x = BatchNormalization(axis = 3,
                           name = 'bn_conv1')(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding = (1, 1), 
                      name = 'pool1_pad')(x)
    x = MaxPooling2D((3, 3), 
                     strides = (2, 2))(x)
    
    #Stage 2
    x = convolutional_block(input_x = x, 
                            kernel_size = 3, 
                            filters = [64, 64, 256], 
                            stage = 2, 
                            block = 'a', 
                            strides = 1)
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 256],
                       stage = 2,
                       block = 'b')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 256],
                       stage = 2,
                       block = 'c')
    
    #Stage 3
    x = convolutional_block(input_x = x, 
                            kernel_size = 3, 
                            filters = [128, 128, 512], 
                            stage = 3, 
                            block = 'a', 
                            strides = 2)
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [128, 128, 512],
                       stage = 3,
                       block = 'b')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [128, 128, 512],
                       stage = 3,
                       block = 'c')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [128, 128, 512],
                       stage = 3,
                       block = 'd')
    
    #Stage 4
    x = convolutional_block(input_x = x, 
                            kernel_size = 3, 
                            filters = [256, 256, 1024], 
                            stage = 4, 
                            block = 'a', 
                            strides = 2)
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [256, 256, 1024],
                       stage = 4,
                       block = 'b')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [256, 256, 1024],
                       stage = 4,
                       block = 'c')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [256, 256, 1024],
                       stage = 4,
                       block = 'd')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [256, 256, 1024],
                       stage = 4,
                       block = 'e')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [256, 256, 1024],
                       stage = 4,
                       block = 'f')
    
    #Stage 5
    x = convolutional_block(input_x = x, 
                            kernel_size = 3, 
                            filters = [512, 512, 2048], 
                            stage = 5, 
                            block = 'a', 
                            strides = 2)
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [512, 512, 2048],
                       stage = 5,
                       block = 'b')
    x = identity_block(input_x = x,
                       kernel_size = 3,
                       filters = [512, 512, 2048],
                       stage = 5,
                       block = 'c')
    
    #Classification Layer
    x = AveragePooling2D((2,2),
                         name = 'avg_pool')(x)
    
    x = Flatten()(x)  #not in Keras
    x = Dense(classes,
             activation = 'softmax',
             name = 'fc' + str(classes))(x)
    
    #create model
    model = Model(x_input, x, name = 'ResNet50')
    
    return model

In [12]:
resnet50_rebuilt = ResNet50(input_shape = (64, 64, 3), classes = 6)
resnet50_rebuilt.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## Try training with this model

In [15]:
#Get some test images (https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [16]:
#compile our model
resnet50_rebuilt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
#train our model
resnet50_rebuilt.fit(X_train, Y_train, epochs = 5, batch_size = 32)

Epoch 1/5
1080/1080 [==============================] - 82s 75ms/step - loss: 1.7010 - accuracy: 0.4361
Epoch 2/5
1080/1080 [==============================] - 70s 65ms/step - loss: 0.8105 - accuracy: 0.7417
Epoch 3/5
1080/1080 [==============================] - 73s 67ms/step - loss: 0.3553 - accuracy: 0.8824
Epoch 4/5
1080/1080 [==============================] - 72s 66ms/step - loss: 0.2794 - accuracy: 0.9037
Epoch 5/5
1080/1080 [==============================] - 74s 69ms/step - loss: 0.2469 - accuracy: 0.9259
